<a href="https://colab.research.google.com/github/EvgeniiOjug/Projects-WB/blob/main/%D0%9F%D0%B0%D1%80%D1%81%D0%B8%D0%BD%D0%B3_WB_%D0%BD%D0%B5_%D0%B1%D1%80%D0%B5%D0%BD%D0%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas
from pandas import ExcelWriter
import openpyxl
import lxml

# info парсер wildberries
# реализовал сбор названия и цены, скидку
# сделал пагинацию
# сделал поиск по введенному значению
# 1.0) сделать поиск по параметру (введенному от пользователя)
# 1.2) для этого надо предоставить выбор пользователю (например вывести список брендов и выбрать определенный)
# доделать пагинацию (в данный момент значение берется из начального экрана, а там могут отображаться не все страницы)

#url = "https://www.wildberries.ru/catalog/0/search.aspx?search=%D0%B2%D0%B8%D0%B4%D0%B5%D0%BE%D0%BA%D0%B0%D1%80%D1%82%D1%8B"
#url = 'https://www.wildberries.ru/catalog/0/search.aspx?search=%D0%B2%D0%B8%D0%B4%D0%B5%D0%BE%D0%BA%D0%B0%D1%80%D1%82%D0%B0&xsearch=true'
#url = 'https://www.wildberries.ru/catalog/0/search.aspx?search=%D0%BD%D0%BE%D1%83%D1%82%D0%B1%D1%83%D0%BA&xsearch=true'
#url = 'https://www.wildberries.ru/catalog/0/search.aspx?search=%D0%BD%D0%BE%D1%83%D1%82%D0%B1%D1%83%D0%BA%D0%B8&xparams=subject%253D2290&xshard=electronic&xfilters=xsubject;dlvr;brand;price;kind;color;wbsize;season;consists&xsearch=true&fbrand=5786'
headers = {
    "Accept": "*/*",
    "Accept-Language": "ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:84.0) Gecko/20100101 Firefox/84.0",
}


def get_html(url, params=None):
    response = requests.get(url, headers=headers, params=params)
    html = response.text
    return html


def get_pages(html):
    """ получаем количество страниц"""
    soup = BeautifulSoup(html, 'lxml')
    pagination = soup.find('span', class_='brand-custom-header__count')
    if ' ' in pagination:
      good_count = soup.find('span', class_='brand-custom-header__count').get_text(strip=True).replace(' ', '').split()[0]
    else:
      good_count = soup.find('span', class_='brand-custom-header__count').get_text(strip=True).split()[0] 
    print(good_count)
    pages = int(good_count) // 100 + 1
    return pages


def get_content(html):
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.find_all('div', class_="product-card")
    cards = []
    for item in items:
        # проверка на наличии скидки, если нет, то поле пустое
        discount = item.find('span', class_='product-card__sale active')
        if discount:
            discount = discount.get_text(strip=True)
        else:
            discount = ''

        cards.append({
            'brand': item.find('strong', class_='brand-name').get_text(strip=True).replace('/', ''),
            'title': item.find('span', class_='goods-name').get_text().split('/')[0],
            'price': int(item.find(class_='lower-price').get_text(strip=True).replace('\xa0', '').replace('₽', '')),
            'discount': discount,
            'link': f'https://www.wildberries.ru{item.find("a", class_="product-card__main").get("href")}',
        })
    return cards


def save_exel(data):
    # сохраняем полученные данные в эксель с помощью dataframe от pandas
    dataframe = pandas.DataFrame(data)
    newdataframe = dataframe.rename(columns={'brand': 'Брэнд', 'title': 'Наименование',
                                             'price': 'Цена',  'discount': 'Скидка',
                                             'link': 'Ссылка'})
    writer = ExcelWriter(f'data.xlsx')
    newdataframe.to_excel(writer, 'data')
    writer.save()
    print(f'Данные сохранены в файл "data.xlsx"')


def parse(url):
    global search
    search = url
    # url ='https://www.wildberries.ru/catalog/sport/vidy-sporta/velosport/velosipedy'
    # url = 'https://www.wildberries.ru/brands/xiaomi/all'
    print(f'Парсим данные с: "{search}"')
    #print(url)
    # try:
    html = get_html(url)
    pages = get_pages(html)
    print(f'Количество страниц: {pages}')
    cards = []
    pages = int(input('Введите количество страниц: '))
    for page in range(1, pages + 1):
        print(f'Парсинг страницы: {page}')
        html = get_html(url, params={'sort': 'popular', 'page': page})
        cards.extend(get_content(html))
    print(f'Всего: {len(cards)} позиций')
    save_exel(cards)
    # except Exception as ex:
    #     print(f'Данные по запросу не найдены')


if __name__ == "__main__":
    parse('https://www.wildberries.ru/brands/-a-and-s-fashion')



Парсим данные с: "https://www.wildberries.ru/brands/-a-and-s-fashion"
19
Количество страниц: 1
